In [1]:
"""
Created on Tue Feb 12 23:34:44 2019
@author: tobia
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData

import os
import glob
from os.path import expanduser
from dotenv import load_dotenv

from pysofar.sofar import SofarApi
from pysofar.spotter import Spotter

In [2]:
%load_ext autoreload
%autoreload 2
from oceanoobsbrasil.others.spotter_data import SpotterData

In [3]:
s = SpotterData()

In [4]:
s.get()

SPOT-0335
No sst data for buoy SPOT-0335 from 2023-06-01 to 2023-06-03
SPOT-0334
No wind data for buoy SPOT-0334 from 2023-06-01 to 2023-06-03
No wave data for buoy SPOT-0334 from 2023-06-01 to 2023-06-03
No sst data for buoy SPOT-0334 from 2023-06-01 to 2023-06-03
SPOT-0746
No wind data for buoy SPOT-0746 from 2023-06-01 to 2023-06-03
No wave data for buoy SPOT-0746 from 2023-06-01 to 2023-06-03
No sst data for buoy SPOT-0746 from 2023-06-01 to 2023-06-03
SPOT-0739
No wind data for buoy SPOT-0739 from 2023-06-01 to 2023-06-03
No wave data for buoy SPOT-0739 from 2023-06-01 to 2023-06-03
No sst data for buoy SPOT-0739 from 2023-06-01 to 2023-06-03
SPOT-0745
No wind data for buoy SPOT-0745 from 2023-06-01 to 2023-06-03
No wave data for buoy SPOT-0745 from 2023-06-01 to 2023-06-03
No sst data for buoy SPOT-0745 from 2023-06-01 to 2023-06-03
SPOT-1272
No sst data for buoy SPOT-1272 from 2023-06-01 to 2023-06-03
SPOT-1293
No sst data for buoy SPOT-1293 from 2023-06-01 to 2023-06-03
SPOT-01

In [5]:
s.result.head()

,name,date_time,lat,lon,wspd,wdir,swvht,tp,wvdir,sst,institution,data_type,flag
0,SPOT-010065,2022-07-01 12:31:01,-55.801550,20.779683,10.0,251,3.624,11.378,264.375,-0.44,sofar,drifter,False
1,SPOT-010065,2022-07-02 10:21:19,-55.544267,21.169650,6.0,268,4.314,11.378,223.872,-0.34,sofar,drifter,False
2,SPOT-010065,2022-07-02 11:21:19,-55.536167,21.195967,10.4,251,4.193,11.378,231.800,-0.34,sofar,drifter,False
3,SPOT-010065,2022-07-03 09:15:01,-55.509783,21.737233,6.4,291,3.295,9.309,255.392,-0.40,sofar,drifter,False
4,SPOT-010065,2022-07-03 10:15:01,-55.513967,21.756583,9.2,268,3.405,10.240,253.806,-0.36,sofar,drifter,False


In [45]:
start_date = (datetime.utcnow() - timedelta(days=3)).strftime('%Y-%m-%d')
end_date = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')

In [48]:
spt_data = s.spotters[15].grab_data(
    limit=100,
    start_date=start_date,
    end_date=end_date,
    include_track=True,
    include_wind=True,
    include_surface_temp_data=True,
    include_frequency_data=False,
    include_directional_moments=True
)

In [54]:
format_date = '%Y-%m-%dT%H:%M:%S.000Z'
wind = pd.json_normalize(spt_data, record_path=['wind'], meta=['spotterId'])
wind.loc[:,'timestamp'] = pd.to_datetime(wind.loc[:,'timestamp'], format = format_date)
wave = pd.json_normalize(spt_data, record_path=['waves'], meta=['spotterId'])
wave.loc[:,'timestamp'] = pd.to_datetime(wave.loc[:,'timestamp'], format=format_date)
sst = pd.json_normalize(spt_data, record_path=['surfaceTemp'], meta=['spotterId'])
sst.loc[:,'timestamp'] = pd.to_datetime(sst.loc[:,'timestamp'], format = format_date)

In [61]:
wave.head()

,significantWaveHeight,peakPeriod,meanPeriod,peakDirection,peakDirectionalSpread,meanDirection,meanDirectionalSpread,timestamp,latitude,longitude,spotterId
0,2.526,10.240,6.890,231.705,29.839,240.352,37.392,2022-06-03 12:57:55,-52.22768,-36.73440,SPOT-010098
1,2.424,10.240,6.773,226.819,35.720,240.867,40.586,2022-06-03 13:57:55,-52.22268,-36.71477,SPOT-010098
2,2.366,10.240,6.760,232.727,40.469,242.105,42.120,2022-06-03 14:57:55,-52.21622,-36.69637,SPOT-010098
3,2.327,9.309,6.591,232.651,40.706,244.882,41.648,2022-06-03 15:57:55,-52.20785,-36.67853,SPOT-010098
4,2.372,9.309,6.694,239.101,41.279,247.922,41.640,2022-06-03 16:57:55,-52.19993,-36.66300,SPOT-010098


In [44]:
s.result

,wspd,wdir,seasurfaceId,latitude,longitude,date_time,swvht,peakPeriod,meanPeriod,peakDirection,peakDirectionalSpread,meanDirection,meanDirectionalSpread,sst,spotter_id
0,6.0,205.0,2.0,-54.79155,10.258633,2022-06-03 12:13:31,None,None,None,None,None,None,None,-0.5,SPOT-010065
1,6.4,188.0,2.0,-54.776617,10.26305,2022-06-03 13:13:31,None,None,None,None,None,None,None,-0.5,SPOT-010065
2,7.6,188.0,2.0,-54.76175,10.266817,2022-06-03 14:13:31,None,None,None,None,None,None,None,-0.5,SPOT-010065
3,6.0,211.0,2.0,-54.7455,10.27225,2022-06-03 15:13:31,3.426,12.799,8.561,288.844,29.711,278.916,60.77,-0.5,SPOT-010065
4,6.8,194.0,2.0,-54.730233,10.276017,2022-06-03 16:13:31,None,None,None,None,None,None,None,-0.54,SPOT-010065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,None,None,None,-58.393517,-27.7207,2022-06-19 20:42:01,None,None,None,None,None,None,None,-0.34,SPOT-010126
980,None,None,None,-57.43875,-25.857183,2022-07-02 13:53:01,None,None,None,None,None,None,None,-0.66,SPOT-010126
981,None,None,None,-57.436333,-25.850267,2022-07-02 14:13:01,None,None,None,None,None,None,None,-0.66,SPOT-010126
982,None,None,None,-57.431833,-25.8387,2022-07-02 14:53:01,None,None,None,None,None,None,None,-0.66,SPOT-010126


In [32]:
s.data[['date_time','latitude','longitude','wspd','wdir','swvht','peakPeriod',
                          'meanDirection', 'sst']]
                          

,date_time,latitude,longitude,wspd,wdir,swvht,peakPeriod,meanDirection,sst
0,2022-06-03 12:13:31,-54.79155,10.258633,6.0,205.0,3.733,12.799,277.879,-0.5
1,2022-06-03 13:13:31,-54.776617,10.26305,6.4,188.0,3.504,12.799,281.835,-0.5
2,2022-06-03 14:13:31,-54.76175,10.266817,7.6,188.0,3.377,12.799,277.61,-0.5
3,2022-06-03 15:13:31,-54.7455,10.27225,6.0,211.0,3.426,12.799,278.916,-0.5
4,2022-06-03 16:13:31,-54.730233,10.276017,6.8,194.0,3.441,11.378,273.241,-0.54
...,...,...,...,...,...,...,...,...,...
755,2022-06-19 20:42:01,None,None,None,None,None,None,None,-0.34
756,2022-07-02 13:53:01,None,None,None,None,None,None,None,-0.66
757,2022-07-02 14:13:01,None,None,None,None,None,None,None,-0.66
758,2022-07-02 14:53:01,None,None,None,None,None,None,None,-0.66


In [4]:
bd = GetData()

In [4]:
bd.get(table='stations', institution=['=', 'simcosta'], data_type=['=', 'buoy'])

,id,name,lat,lon,data_type,institution,url
0,897,RJ3,-22.9831,-43.1745,buoy,simcosta,3
1,898,RS2,-32.1346,-52098.0000,buoy,simcosta,6
2,899,RS5,-32296.0000,-52.0239,buoy,simcosta,10
3,900,RJ4,-22.9715,-43.1522,buoy,simcosta,12
4,901,RS3,-32195.0000,-52.0794,buoy,simcosta,13
5,902,RS4,-32.2454,-52.0954,buoy,simcosta,14
6,903,BA1,-12.9895,-37.4582,buoy,simcosta,15


In [8]:
df = bd.get(table='data_stations', station_id=['in', [902]])

In [9]:
df.shape

(94, 22)

In [7]:
bd.post(table='data_stations', df=df)

DELETE FROM data_stations WHERE true AND station_id in (902) AND date_time >= '2021-09-06 00:21:00'


In [5]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.buoys.simcosta import Simcosta
from oceanoobsbrasil.tides.simcosta import SimcostaTide

In [6]:
s = SimcostaTide()

In [7]:
s.get()

Nao ha dados para essa boia
Nao ha dados para essa boia
Nao ha dados para essa boia


In [8]:
s = Simcosta()

In [9]:
s.get()

210
Nao ha dados para essa boia
504
Nao ha dados para essa boia
15
ok
3
ok
12
ok
14
Nao ha dados para essa boia
13
Nao ha dados para essa boia
510
ok
6
Nao ha dados para essa boia


In [5]:
from oceanoobsbrasil.buoys.pnboia import Pnboia

In [7]:
p = Pnboia()
p.get()

alcatrazes
Dados inseridos
bacia_de_santos
'date_time'
abrolhos
Dados inseridos
almirantado
'date_time'
noronha
Dados inseridos
mexilhão
'date_time'
pinguim
'date_time'
keller
'date_time'
bacia_de_campos
'date_time'
imbituba
Dados inseridos


In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.